Symulacja populacji dorosłych osób i analiza:

- BMI
- Stylu życia
- czynników ryzyka zdrowotnego

In [7]:
import numpy as np

N = 2000

# Dane:

wiek = np.random.uniform(18, 80, size = N)

wzrost = np.random.normal(170, 10, size = N) 

waga = np.random.normal(75, 15, size = N)

aktywnosc = np.random.choice(["niska", "średnia", "wysoka"], size = N)

palenie = np.random.choice(["tak", "nie"], size = N, p=[0.25, 0.75])

In [ ]:
# ETAP 2 - Obliczenie NumPy (Maski)

BMI = (waga / (wzrost/2)**2)

# Nadwaga bmi >= 25
nadwaga = (BMI >= 25)
print(f"Nadwaga: \n{nadwaga}")

# Otyłość -> BMI >= 30
otylosc = (BMI >= 30)
print(f"\nOtyłość: \n{otylosc}")

# Niska aktywność
niska_aktywnosc = aktywnosc == "niska"
print(f"\nNiska aktywność: \n{niska_aktywnosc}")

# palacze 
palacze = palenie == "tak"
print(f"\nPalacze: \n{palacze}")



Nadwaga: 
[False False False ... False False False]

Otyłość: 
[False False False ... False False False]

Niska aktywność: 
[False False False ... False  True  True]

Palacze: 
[False False False ... False False False]
